## MOUNT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/MSc_project/.MAIN

Mounted at /content/drive
/content/drive/My Drive/MSc_project/.MAIN


## PIPS

In [2]:
!pip install preprocessor
!pip install tweepy==4.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4477 sha256=1411082420dd9cbe66bf22ca39a865bb12a151fcd61ed8574b6ab37871c3cac1
  Stored in directory: /root/.cache/pip/wheels/0e/b7/36/aa37256db62b4bfd35a6f1b5536e9ba843f257b79dcbf3d5f1
Successfully built preprocessor
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


## IMPORTS 

In [3]:
import pandas as pd
import numpy as np
import tweepy
import json
import os
import os.path
import time
import multiprocessing

## GRAB FEEDS FUNCTION

In [41]:
from matplotlib import collections
from psutil import users
import tweepy
from tweepy import OAuthHandler
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
from datetime import datetime
import json
import time
import glob
import os
from collections import defaultdict
import configparser
from copy import deepcopy
import multiprocessing
from joblib import Parallel, delayed
import time
import os.path
import random


class crawler(object):
    def __init__(self, hashtag,file_names,num_keys): 

        self.hashtag = hashtag
        self.sep = os.path.sep  
        self.file_names = file_names
        self.num_keys = num_keys
        self.ms_path = f'tweets/{hashtag}'
        self.path = f'tweets/{hashtag}/100_feeds'
        if not os.path.exists(self.path):
            os.makedirs(self.path)


###########################################################
###########################################################
# API KEY FUNCTIONS
###########################################################
###########################################################


    def get_api_list(self,filename,num_keys):
        settings_file = f"apikeys{self.sep}{filename}"
        # Read config settings
        config = configparser.ConfigParser()
        config = configparser.ConfigParser(interpolation=None)
        config.readfp(open(settings_file))

        # Create API objects for each of the API keys
        # 1-based indexing of config file
        start_idx = 1
        end_idx = num_keys
        num_api_keys = end_idx - start_idx + 1

        apis = []

        print("Creating api objects for {} API keys".format(num_api_keys))
        for api_idx in range(start_idx, end_idx + 1):
            consumer_key = config.get('API Keys ' + str(api_idx), 'consumer_key')
            consumer_secret = config.get('API Keys ' + str(api_idx), 'consumer_secret')
            access_token_key = config.get('API Keys ' + str(api_idx),
                                        'access_token')
            access_token_secret = config.get('API Keys ' + str(api_idx),
                                            'access_secret')

            # Connect to Twitter API
            try:
                auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
                auth.set_access_token(access_token_key, access_token_secret)
                api = tweepy.API(auth, wait_on_rate_limit=True)
            except Exception as e:
                print("Error while creating API object: " + str(e))
                continue
            else:
                apis.append(api)
        print(f'found {len(apis)} apis from {filename}')
        idx = []
        for i, apii in enumerate(apis):
            try: 
                a = apii.verify_credentials()
            except tweepy.errors.TweepyException as e:
                idx.append(i)
                # print(i+1)
                # print(e)
        
        working_api = np.delete(apis,idx)
        print(f'{len(working_api)} api work')
        return working_api.tolist()

    def make_api(self):
        consumer_key = '3rJOl1ODzm9yZy63FACdg'
        consumer_secret = '5jPoQ5kQvMJFDYRNE8bQ4rHuds4xJqhvgNJM4awaE8'

        tokens="1273621486201450497-2KsGK9JgSSdabGYxFlDNB214MXwi35:kbLULiG4k7SvUlwFkr0p6ESTdvARIThfmI1lQ5GuQmH8s,"
        # tokens+="1186873796982128641-RHYpdyKJIfSX6KtRfA6k73emHMOhBY:Zth72rrtcZGfqJ4B9TK8dH1RhS8HC6KOlTDqA3bJnMnGL," DOES NOT WORK
        tokens+="1186715756437868546-U9LOFzfSEk4tTOYHoKn7X08vWFm3An:kWWnOy6JLbHI9nG8GihGIjB1SeH5nJE8oqi4KiRIiHB2B,"
        tokens+="1186707518799650816-KOvTi7HOrZ1KiVRXyAvNjpJofF3lY8:EsXAVrtyqkekdmtr0GcZCiVk9RwAJC4v9Kj3fo0RWPpgp,"
        tokens+="1186873046726041600-rNJoDJulul6m5GQUbXUG7zNfpnJywj:OsR7WIHZ9PEdlmIE7K36V1CrM3OiMKIzAboNXMdEJm1UJ,"
        tokens+="1186872285782794240-X4x7t8KRaf3Ce3oNI2ka7DO7rapVys:VUyNnBIRr6oRpBr10lbEF0MzlGz6eJJW3STUWOGyXgVbr,"
        tokens+="1261898258739126273-Dw4HYHPkKj82V4RcgYhpxLsnJdMALV:384AYTeZqRykw5ADdVXhi1BXZadOuflhzOiKljh1Pt0bi,"
        tokens+="1261898258739126273-Dw4HYHPkKj82V4RcgYhpxLsnJdMALV:384AYTeZqRykw5ADdVXhi1BXZadOuflhzOiKljh1Pt0bi,"
        tokens+="1261901714564583424-odKz6Xonlzg1QVg32o6hdqXbGAZ6nt:lx7hysS64y7kyc0pr4IMdqXNJr8yqbgY9vZ7qhnL6ldGY,"
        tokens+="1261905369485373440-mxYEQlU9tEjFvFClg3YTZU2Krh4ezO:mcvotgeJvuOgkJb5pXejP9qzjDBcALZSHqGJesH7FzOOs,"
        tokens+="1261908827961585664-YkhHP2z6XczyTwBt8pT7IYmYw4XFTN:OQJhcBCQiHWCOS1lcUHlZp8Minc7drmpW861tFNwweABC,"
        tokens+="1261911771528364032-uFSYzCtvZeQPLvSIR2NZM3MwYnPDx9:gjLJUDRkiC6qVhh6zN8czDLzcE3Bnhge9Sgq7QIQsRfZp,"
        tokens+="1261913654967992321-dq90nrKd8OTIAiBwK4rmDPR1hvBwTb:6lL8f15qO9DOMczUzImxwUjMmDvPHlQcd3eJBtUzQ53ck,"
        tokens+="1261918659942871040-ThZsmIIeIWLfTuhNB8uD3HORhunMVS:D3QNGphJ1R870yzBV41QSvezqS6rysRVgEG1RU3c7tH30,"
        tokens+="1261929369691283456-0CdW3UgD4ONxFCshwmBqroq3m8BFG7:YpeYZ3hWRNkFOzfoJGiIPXnW65vzul5e7ofDN1q3zZErV,"
        tokens+="1261930395513192448-14EJHzMXDGzDN65NAx9HLXaz8Y1OHf:97qLbZ53tPQl5xwmNPDBlADBrSlZZeXd659N5XXDG2O5E,"
        tokens+="1261931615837581317-Bdgkok64OnA0HWB9q3NZfFF12FpBjD:qeYMydCMxt2ctLOkfNV3yix6erJoxti7NiOScNCQ0zHf6,"
        tokens+="1261932804046114816-5s1MVYfHsfwjxz1pZ2jhLE3jgO0enO:8XmjhdkKTakxF9a3ZKCMYJzlNUb9ARChEZ74XH91qZVqg,"
        tokens+="1259732232433856512-knm0EqhoG7tyEjTsgIQc6974cEy976:lavQjBjsOdZgUy5U64ZtK8YYdfwQeknl9Oy2CP7nJ3uE9,"
        tokens+="1262372803174531072-uyPlAvb0BHwSuB1kgPOroGMyjFPtzj:Bsnfo8WTYNqEzr5S9fW9gzfWFc9E2ZqM0wXwUGIUgSwPo,"
        tokens+="1262374702003044353-GCVqUDfNPkRGnBzmKHTPeZmmsBHjxx:pzqueROwD89WH3hZyr7qDV4ftnu8OPZM3OfhyNSM4pi9T,"
        tokens+="1261903040946171905-mLBFRyOpShKunFrxk5o5DDoJERTEE1:Osvx77SI7qS0qIIYqlI43eLOCOS6UL1e3XSYcUyrYpCHz"

        connections=[]

        for tk in tokens.split(","):
            token=tk.split(":")
            auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret,token[0], token[1])
            api = tweepy.API(auth, wait_on_rate_limit=True)
            # auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
            # auth.set_access_token(token[0], token[1])
            # api = tweepy.API(auth, wait_on_rate_limit=True)
            
            connections.append(api) 


        print(f'\nfound {len(connections)} apis from juans stuff')
        idx = []
        for i, apii in enumerate(connections):
            try: 
                a = apii.verify_credentials()
            except tweepy.TweepyException as e:
                idx.append(i)
                # print(e)
        
        working_api = np.delete(connections,idx)
        print(f'{len(working_api)} api work')
        return working_api.tolist()

    ### HELPER FUNCTIONS

    def load_ms_cases(self):
        path = f'{self.ms_path}/{self.hashtag}_ms_cases.json'
        with open(path) as jf:
            data = json.load(jf)
        return data

    def get_feeds_db(self):
        jsons = [pos_json for pos_json in os.listdir(self.path) if pos_json.endswith('.json')]
        all_js = {}
        for file in jsons:
            with open(os.path.join(f'{self.path}/' + file)) as jf:
                all_js = { **all_js, **json.load(jf) }
        print(f'pulled data on {len(all_js)} users')
        return all_js

    def find_all_users(self,data):

        users = []
        for key,value in data.items():
            users.append(value['user-id'])
            infector = value['infector-info']
            i = [k for k in infector]
            inf = infector[i[0]]
            users.append(inf['user-id'])
            for informer in value['informers-data']:
                users.append(informer['user-id'])
        unique_users = list(set(users))
        return unique_users

    @staticmethod
    def process_tweet(tweet):
        d = {'id':tweet['id_str'], 'tweet-text': tweet['text'],'created_at': tweet['created_at'],
        'retweet_count': tweet['retweet_count'], 'favourite_count':tweet['favorite_count']}
        return d

    def check_already_done(self,users):

        users_done_data = self.get_feeds_db()
        users_done_ids = list( users_done_data.keys() )

        print(f'already have feeds of {len(users_done_ids)} users')

        not_done = [user for user in users if user not in users_done_ids ]

        print(not_done)

        print(f'now getting feeds of remaining {len(not_done)} users')

        return not_done


    #######
    #### MAIN FUNCTION
    #######


    def get_all_feeds(self):

        print(f'working on hashtag {self.hashtag}')

        juan_api = self.make_api()

        everyone_api = self.get_api_list(self.file_names[0],self.num_keys[0])

        just_my_api = self.get_api_list(self.file_names[1],self.num_keys[1])

        all_api = just_my_api + juan_api + everyone_api

        self.all_api = all_api
        num_api = len(all_api)
        print(f'total working api is {num_api}')
        #  we will keep track of the api that we use

        ################################################################################################################
        ########################################################
        ### PROCESS INTO NECESSARY FORMAT

        data = self.load_ms_cases()

        print('found unique users of this set')
        users = self.find_all_users(data)


        ### CHECKING FOR USERS THAT WE HAVE NOT EXTRACTED FEEDS FOR

        users = self.check_already_done(users)

        print(f'Getting the feeds for each unique user in the hashtag {self.hashtag}')
        print('begin timing...\n')

        start = time.perf_counter()
        feeds = {}
        count_users_pulled = 0 # counter count the number of accounts we request to pull in this batch
        num_api = len(all_api)-1
        api_idx = 0

        for user in users:

            try:
                api_idx = np.random.randint(0,num_api)
                print(f'using api number {api_idx}')
                pages = all_api[api_idx].user_timeline(user_id=user, count = 100)
                count_users_pulled += 1
                ## process each tweet
                processed_tweets = [self.process_tweet(tweet._json) for tweet in pages]
                feeds.update({user:processed_tweets} )
                stamp = datetime.today().strftime('%Y%m%d_%H%M%S')
                rn = np.random.randint(0,99999999)
                s_path = f'{self.path}/feeds_{stamp}_{rn}.json'
                print(feeds)
                with open(s_path, 'w') as fp:
                    json.dump(feeds,fp)
                feeds.clear()

            except tweepy.TweepyException as e: # we've got an error with the tweepy API
                print(e) 
                print(f'get rid of api number {api_idx}')
                del all_api[api_idx]
                num_api -=1 # working with one less api

        fin = time.perf_counter()
        print(f'finished extracting tweets of everyone in {(fin-start)/3600} hours')

        stamp = datetime.today().strftime('%Y%m%d_%H%M%S')
        s_path = f'{self.path}/feeds_{stamp}.json'
        with open(s_path, 'w') as fp:
            json.dump(feeds,fp)

## Run Script

In [42]:
# hashtags = ['avengers','blm','borisjohnson','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']

hashtags = ['avengers','blm','borisjohnson','brexit']

# hashtags = ['borisjohnson','brexit']

# BLM NOT WORKING

key_paths = ['apikeys.txt', 'apikeys2.txt']

num_of_keys = [45,83]

for hashtag in hashtags:

    c = crawler(hashtag,key_paths, num_of_keys)

    c.get_all_feeds()
    print(f'DONE {hashtag}')
    print('')

working on hashtag avengers

found 21 apis from juans stuff
21 api work
Creating api objects for 45 API keys
found 45 apis from apikeys.txt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.


30 api work
Creating api objects for 83 API keys
found 83 apis from apikeys2.txt
76 api work
total working api is 127
found unique users of this set
pulled data on 1508 users
already have feeds of 1508 users
['1259659513239449603', '889042890', '1531729381403467777', '1348065530041819137', '3777549741', '750952610388213760', '1552291452159410176', '1424688336812138500', '1341377110590410754', '1317123519000375297', '1513189098118389765', '1054384005657767936', '1334889034657624066', '1094518936589090816', '1541146252703932423']
now getting feeds of remaining 15 users
Getting the feeds for each unique user in the hashtag avengers
begin timing...

using api number 114
401 Unauthorized
Not authorized.
get rid of api number 114
using api number 84
401 Unauthorized
Not authorized.
get rid of api number 84
using api number 39
401 Unauthorized
Not authorized.
get rid of api number 39
using api number 43
401 Unauthorized
Not authorized.
get rid of api number 43
using api number 23
401 Unauthor